In [1]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [2]:
from data import documents

/Users/richardlai/miniconda3/envs/AIE4-midterm2/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/richardlai/miniconda3/envs/AIE4-midterm2/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/richardlai/miniconda3/envs/AIE4-midterm2/lib/python3.11/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Py

Adding file**** /Users/richardlai/Documents/MyProjects/Education/AI-Makerspace-Cohort-4/midterm/ai-policy-rag/src/vectorstore/pdfs/Blueprint-for-an-AI-Bill-of-Rights.pdf
Adding file**** /Users/richardlai/Documents/MyProjects/Education/AI-Makerspace-Cohort-4/midterm/ai-policy-rag/src/vectorstore/pdfs/NIST.AI.600-1.pdf


In [3]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)
question_generation_chain = qa_prompt_template | qa_chat_model

In [4]:
import tqdm
import uuid

def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}
  for document in tqdm.tqdm(documents):
    questions_generated = question_generation_chain.invoke({"context": document.page_content, "n_questions": n_questions})
    for question in questions_generated.content.split("\n"):
      question_id = str(uuid.uuid4())
      questions[question_id] = "".join(question.split(".")[1:]).strip()
      relevant_docs[question_id] =  [document.metadata["id"]]       #[document.metadata["id"]]
  return questions, relevant_docs

In [5]:
training_split_documents = documents[:100]
val_split_documents = documents[100:115]
test_split_documents = documents[115:]

In [6]:
training_questions, training_relevant_contexts = create_questions(training_split_documents, n_questions=2)
val_questions, val_relevant_contexts = create_questions(val_split_documents, n_questions=2)
test_questions, test_relevant_contexts = create_questions(test_split_documents, n_questions=2)

100%|██████████| 22/22 [00:23<00:00,  1.09s/it]


In [7]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

In [8]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

In [9]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [10]:
BATCH_SIZE = 16

corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [11]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [12]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [13]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

EPOCHS = 5

In [14]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

 77%|███████▋  | 50/65 [1:05:22<12:59, 52.00s/it]   

{'eval_cosine_accuracy@1': 0.6666666666666666, 'eval_cosine_accuracy@3': 0.9, 'eval_cosine_accuracy@5': 0.9666666666666667, 'eval_cosine_accuracy@10': 1.0, 'eval_cosine_precision@1': 0.6666666666666666, 'eval_cosine_precision@3': 0.3, 'eval_cosine_precision@5': 0.19333333333333338, 'eval_cosine_precision@10': 0.10000000000000003, 'eval_cosine_recall@1': 0.6666666666666666, 'eval_cosine_recall@3': 0.9, 'eval_cosine_recall@5': 0.9666666666666667, 'eval_cosine_recall@10': 1.0, 'eval_cosine_ndcg@10': 0.8355508604376777, 'eval_cosine_mrr@10': 0.7816666666666666, 'eval_cosine_map@100': 0.7816666666666666, 'eval_dot_accuracy@1': 0.6666666666666666, 'eval_dot_accuracy@3': 0.9, 'eval_dot_accuracy@5': 0.9666666666666667, 'eval_dot_accuracy@10': 1.0, 'eval_dot_precision@1': 0.6666666666666666, 'eval_dot_precision@3': 0.3, 'eval_dot_precision@5': 0.19333333333333338, 'eval_dot_precision@10': 0.10000000000000003, 'eval_dot_recall@1': 0.6666666666666666, 'eval_dot_recall@3': 0.9, 'eval_dot_recall@5'

100%|██████████| 65/65 [1:21:12<00:00, 74.96s/it] 

{'train_runtime': 4872.5626, 'train_samples_per_second': 0.205, 'train_steps_per_second': 0.013, 'train_loss': 2.6060093806340143, 'epoch': 5.0}


In [18]:
from huggingface_hub import notebook_login, login
token="..."
login(token=token, add_to_git_credential=True)
model.push_to_hub("rgtlai/ai-policy-ft")

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/richardlai/.cache/huggingface/token
Login successful


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
model.safetensors: 100%|████

'https://huggingface.co/rgtlai/ai-policy-ft/commit/ffe92c92d36d96913ac46855a027ac85fb8396a5'